## Plot CRT On Board Survey frequency

In [6]:
process_scenarios = [
    'E2.17.3__BY_2019',
    'E2.14.4__BY_2019'
]
model_id_for_crt_distances_for_obs = "E2.14.4__BY_2019" # must be in list above. To make TDM and OBS more comparable, the TDM distance matrix of a model is used for the CRT distance.

```
OMX conversion Voyager syntax: 
CONVERTMAT FROM='skm_d8_Pk.mtx', TO='skm_d8_Pk.omx', FORMAT=OMX COMPRESSION=6
CONVERTMAT FROM='skm_d8_Ok.mtx', TO='skm_d8_Ok.omx', FORMAT=OMX COMPRESSION=6
CONVERTMAT FROM='skm_w8_Pk.mtx', TO='skm_w8_Pk.omx', FORMAT=OMX COMPRESSION=6
CONVERTMAT FROM='skm_w8_Ok.mtx', TO='skm_w8_Ok.omx', FORMAT=OMX COMPRESSION=6
```

In [7]:
import os
import pandas as pd
import numpy as np
import openmatrix as omx
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import ipywidgets as widgets
import numpy as np

import _global_scripts as gs

In [8]:
# get data from csv now
models_df = pd.read_csv("data-scenario/models.csv")

summary_models = models_df[
    models_df["model_display"] & models_df["model_RID"].notna()
]

data_paths = {}

for _, row in summary_models.iterrows():
    model_id = row["model_id"]
    data_paths[model_id] = f"data-scenario/{row['scenario_id']}/{row['model_name']}"
data_paths

{'v910-official__BY_2019': 'data-scenario/v910-official/BY_2019',
 'v910-official__RTP_2050': 'data-scenario/v910-official/RTP_2050',
 'E2.14.4__BY_2019': 'data-scenario/E2.14.4/BY_2019',
 'E2.16.2__BY_2019': 'data-scenario/E2.16.2/BY_2019',
 'E2.16.3__BY_2019': 'data-scenario/E2.16.3/BY_2019',
 'E2.17.3__BY_2019': 'data-scenario/E2.17.3/BY_2019',
 'E2.18.2__BY_2019': 'data-scenario/E2.18.2/BY_2019',
 'E2.18.2__RTP_2050': 'data-scenario/E2.18.2/RTP_2050',
 'E2.18.2__RTP_MedDistAdj_2050': 'data-scenario/E2.18.2/RTP_MedDistAdj_2050',
 'E2.18.3__BY_2019': 'data-scenario/E2.18.3/BY_2019',
 'E2.18.4__BY_2019': 'data-scenario/E2.18.4/BY_2019'}

In [9]:
# Define transit skim files
transit_skim_loc = [
    r"skm_d8_Pk.omx", r"skm_d8_Ok.omx",
    r"skm_w8_Pk.omx", r"skm_w8_Ok.omx"
]

# Define invalid TAZs
invalid_model_taz_list = np.arange(3547, 3601)

skimloc_mapping = {
    ('PK', 'dCRT'): 0,
    ('PK', 'wCRT'): 2,
    ('OK', 'dCRT'): 1,
    ('OK', 'wCRT'): 3
}
access_mapping = {'dCRT': 'Drive', 'wCRT': 'Walk'}

# Filter OBS Data
CRTpkok = gs.obs_df[gs.obs_df['Linked_Mode'] == 8]
columns = ['p_TAZID', 'a_TAZID', 'Ac_Mode_Model', 'PK_OK', 'linked_weight_adj', 'Veh_Cat3p', 'Purp5_text']
CRTpkok = CRTpkok[columns]
CRTpkok.loc[CRTpkok['Veh_Cat3p'] == 3, 'Veh_Cat3p'] = 2
CRTpkok.rename(columns={'linked_weight_adj':'trips_count'}, inplace=True)


# Function to convert model data into DataFrame
def create_model_mtx_to_df(trips_file_name, crtime_file_name, trips_mtx_name, crtime_mtx_name='D8', delZero=True):
    trips_file = omx.open_file(trips_file_name)
    trips_mtx = np.array(trips_file[trips_mtx_name])
    crtime_file = omx.open_file(crtime_file_name)
    crtime_mtx = np.array(crtime_file[crtime_mtx_name])

    trips_df = pd.DataFrame(pd.DataFrame(trips_mtx).stack()).rename({0: 'trips_count'}, axis=1)
    crtime_df = pd.DataFrame(pd.DataFrame(crtime_mtx).stack()).rename({0: 'cr_travel_distance'}, axis=1)

    model_df = pd.concat([trips_df, crtime_df], axis=1).reset_index().rename(
        {'level_0': 'p_TAZID', 'level_1': 'a_TAZID'}, axis=1)

    model_df['p_TAZID'] += 1
    model_df['a_TAZID'] += 1
    model_df = model_df[~model_df['p_TAZID'].isin(invalid_model_taz_list) & ~model_df['a_TAZID'].isin(invalid_model_taz_list)]

    # **Filter out rows where trips_count == 0**
    if delZero: 
        model_df = model_df[model_df['trips_count'] > 0]

    return model_df

# Dictionary to store results from all models
model_results = {}

# Iterate through model versions
for version, data_path in data_paths.items():
    if version not in process_scenarios:
        continue
    print("")
    print(f"Processing Model Version: {version}")

    for purpose in gs.purposes:
        print(f"...{purpose}", end="")
        for period in gs.periods:
            print(f" {period}", end="")
            if purpose=='HBC' and period=='OK':
                print(f" skip", end="")
                continue
            for access in gs.accesses:
                print(f" {access}", end="")

                trips_file_name=os.path.join(data_path, f"{purpose}_trips_allsegs_{period}.omx"),
                crtime_file_name=os.path.join(data_path, transit_skim_loc[skimloc_mapping.get((period, access), None)]),

                access_mapped = access_mapping.get(access, 'Unknown')  # Returns 'Drive'

                model_df = create_model_mtx_to_df(
                    trips_file_name=trips_file_name[0],
                    crtime_file_name=crtime_file_name[0],
                    trips_mtx_name=access,
                    delZero = True
                )

                model_df['trips_count'] /= 100
                model_df = model_df[model_df['cr_travel_distance'] > 0]

                key = f"{purpose}_{access}_{period}_{version}"
                model_results[key] = model_df.copy()

                if version == model_id_for_crt_distances_for_obs:
                    obs_temp_df = create_model_mtx_to_df(
                        trips_file_name=trips_file_name[0],
                        crtime_file_name=crtime_file_name[0],
                        trips_mtx_name=access,
                        delZero=False
                    )
                    obs_temp_df.drop(columns=['trips_count'], inplace=True)
                    obs_temp_df = obs_temp_df[obs_temp_df['cr_travel_distance'] > 0]

                    obs_trips_df = CRTpkok[(CRTpkok['Purp5_text']==purpose) & (CRTpkok['PK_OK']==period) & (CRTpkok['Ac_Mode_Model']==access_mapped)]
                    obs_trips_df = obs_trips_df[['p_TAZID','a_TAZID','trips_count']].copy()
                    df = pd.merge(obs_temp_df, obs_trips_df, on=['p_TAZID','a_TAZID'])

                    key = f"{purpose}_{access}_{period}_OBS"
                    model_results[key] = df.copy()

print("")
print("Done!")


Processing Model Version: E2.14.4__BY_2019
...HBW PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip...NHB PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT
Processing Model Version: E2.17.3__BY_2019
...HBW PK dCRT wCRT OK dCRT wCRT...HBC PK dCRT wCRT OK skip...NHB PK dCRT wCRT OK dCRT wCRT...HBO PK dCRT wCRT OK dCRT wCRT
Done!


In [10]:
# Create first set of dropdowns
model_dropdown_1 = widgets.Dropdown(
    options=['OBS'] + process_scenarios,
    value=process_scenarios[0],
    description="Model 1:",
    layout=widgets.Layout(width="50%")
)

purpose_dropdown_1 = widgets.Dropdown(
    options=gs.purposes,
    value=gs.purposes[0],
    description="Purpose 1:"
)

period_dropdown_1 = widgets.Dropdown(
    options=gs.periods,
    value=gs.periods[0],
    description="Period 1:"
)

access_dropdown_1 = widgets.Dropdown(
    options=gs.accesses,
    value=gs.accesses[0],
    description="Access 1:"
)

# Create second set of dropdowns
model_dropdown_2 = widgets.Dropdown(
    options=['OBS'] + process_scenarios,
    value='OBS',
    description="Model 2:",
    layout=widgets.Layout(width="50%")
)

purpose_dropdown_2 = widgets.Dropdown(
    options=gs.purposes,
    value=gs.purposes[0],
    description="Purpose 2:"
)

period_dropdown_2 = widgets.Dropdown(
    options=gs.periods,
    value=gs.periods[0],
    description="Period 2:"
)

access_dropdown_2 = widgets.Dropdown(
    options=gs.accesses,
    value=gs.accesses[0],
    description="Access 2:"
)



def weighted_median(values, weights):
    """Compute the weighted median of a dataset."""
    sorted_indices = np.argsort(values)
    sorted_values = np.array(values)[sorted_indices]
    sorted_weights = np.array(weights)[sorted_indices]

    cumulative_weight = np.cumsum(sorted_weights)
    total_weight = np.sum(sorted_weights)

    median_idx = np.searchsorted(cumulative_weight, total_weight / 2)
    return sorted_values[median_idx]

def update_chart(models_1, purpose_1, period_1, access_1, models_2, purpose_2, period_2, access_2):

    output.clear_output()  # Clear previous output before displaying new content
    global firstTime
    if firstTime:

        key1 = f"{purpose_1}_{access_1}_{period_1}_{models_1}"
        key2 = f"{purpose_2}_{access_2}_{period_2}_{models_2}"

        # Filter data for both selections
        filtered_df_1 = model_results[key1].copy()
        filtered_df_2 = model_results[key2].copy()

        if filtered_df_1.empty and filtered_df_2.empty:
            print("No data available for the selected criteria.")
            return

        # Compute weighted medians
        median_1 = weighted_median(filtered_df_1['cr_travel_distance'], filtered_df_1['trips_count'])
        median_2 = weighted_median(filtered_df_2['cr_travel_distance'], filtered_df_2['trips_count'])

        filtered_df_1['key'] = 1
        filtered_df_2['key'] = 2

        global_min = min(filtered_df_1['cr_travel_distance'].min(), filtered_df_2['cr_travel_distance'].min())
        global_max = max(filtered_df_1['cr_travel_distance'].max(), filtered_df_2['cr_travel_distance'].max())

        bins = range(int(global_min), int(global_max) + 2, 5)

        fig, ax = plt.subplots(figsize=(10, 6))              

        ### Plotting the weighted distance frequency distribution
        sns.histplot(
            data=filtered_df_1,
            x='cr_travel_distance', 
            weights='trips_count', 
            bins=bins, 
            stat='density',
            ax=ax, 
            kde=True,
            kde_kws={"bw_adjust": 3.0},
            color='darkorange',
            label=f"{key1} (Weighted Median: {median_1:.2f})",
            alpha=0.35)
        
        sns.histplot(
            data=filtered_df_2, 
            x='cr_travel_distance', 
            weights='trips_count', 
            bins=bins, 
            stat='density',
            ax=ax, 
            kde=True,
            kde_kws={"bw_adjust": 3.0},
            color='dodgerblue',
            label=f"{key2} (Weighted Median: {median_2:.2f})",
            alpha=0.35)

        ### Adding vertical lines for weighted medians
        ax.axvline(median_1, color='darkorange', linestyle='dashed', linewidth=2)
        ax.axvline(median_2, color='dodgerblue', linestyle='dashed', linewidth=2)

        ### Adding labels and title
        ax.set_xlabel('CRT Distance')
        ax.set_ylabel('Frequency')
        ax.set_title(f"{key1} vs {key2}")

        ### Adding legend
        ax.legend(title='Legend')

        # Display the plot
        with output:
            plt.show() 
    
    else:
        firstTime = True


# Set up a global variable to track whether the widgets have been changed
firstTime = False

# create output widget to display filtered DataFrame
output = widgets.Output()
hbox1 = widgets.HBox([model_dropdown_1, purpose_dropdown_1, period_dropdown_1, access_dropdown_1])
hbox2 = widgets.HBox([model_dropdown_2, purpose_dropdown_2, period_dropdown_2, access_dropdown_2])
vbox = widgets.VBox([hbox1, hbox2])

# create interactive widget
interactive_output = widgets.interactive_output(update_chart, 
   {'models_1': model_dropdown_1, 
    'purpose_1': purpose_dropdown_1, 
    'period_1': period_dropdown_1, 
    'access_1': access_dropdown_1,
    'models_2': model_dropdown_2, 
    'purpose_2': purpose_dropdown_2, 
    'period_2': period_dropdown_2, 
    'access_2': access_dropdown_2})

display(vbox)
display(interactive_output)
display(output)

Output()

Output()